In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Verse Generation Demo\n",
    "\n",
    "This notebook demonstrates how to generate new verses in the style of Truyện Kiều using the trained language models."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import torch\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from IPython.display import display, HTML\n",
    "\n",
    "from src.verse_generator import KieuVerseGenerator\n",
    "from src.cultural_context import CulturalContextEnhancer"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Trained Models\n",
    "\n",
    "We'll load the previously trained models for verse generation."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check available device\n",
    "device = 'cuda' if torch.cuda.is_available() else 'cpu'\n",
    "print(f\"Using device: {device}\")\n",
    "\n",
    "# Choose model type based on available hardware\n",
    "model_type = 'lstm' if device == 'cuda' else 'ngram'\n",
    "model_path = f'../models/kieu_{model_type}.pkl'\n",
    "\n",
    "try:\n",
    "    # Load the generator\n",
    "    generator = KieuVerseGenerator(model_path, model_type)\n",
    "    print(f\"Loaded {model_type.upper()} model from {model_path}\")\n",
    "    \n",
    "    # Load cultural context enhancer\n",
    "    enhancer = CulturalContextEnhancer()\n",
    "    print(\"Loaded cultural context enhancer\")\n",
    "    \n",
    "except FileNotFoundError:\n",
    "    print(f\"Error: Model file {model_path} not found. Please run the training notebook first.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Basic Verse Generation\n",
    "\n",
    "Generate verses starting with specific phrases."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def display_generated_verses(verses, prompt=None):\n",
    "    \"\"\"Display generated verses with HTML formatting\"\"\"\n",
    "    html = \"<div style='background-color:#f9f9f9; padding:15px; border-radius:5px;'>\"\n",
    "    \n",
    "    if prompt:\n",
    "        html += f\"<h3>Generated verses from: '{prompt}'</h3>\"\n",
    "    else:\n",
    "        html += f\"<h3>Generated verses:</h3>\"\n",
    "    \n",
    "    for i, verse in enumerate(verses, 1):\n",
    "        html += f\"<p><b>{i}.</b> {verse}</p>\"\n",
    "    \n",
    "    html += \"</div>\"\n",
    "    display(HTML(html))\n",
    "\n",
    "# Generate verses from classic starting phrases\n",
    "prompts = [\n",
    "    \"Trăm năm\",  # Famous opening\n",
    "    \"Tình duyên\",  # Love and fate\n",
    "    \"Vầng trăng\",  # Moon imagery\n",
    "    \"Mây trôi\"    # Natural imagery\n",
    "]\n",
    "\n",
    "for prompt in prompts:\n",
    "    verses = generator.generate_verse(prompt, num_samples=3)\n",
    "    display_generated_verses(verses, prompt)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Interactive Verse Generation\n",
    "\n",
    "Let's create an interactive function to generate verses from user input."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from ipywidgets import widgets, interact, interactive, fixed, interact_manual\n",
    "\n",
    "def generate_interactive(prompt, num_samples=3, max_length=50):\n",
    "    verses = generator.generate_verse(prompt, num_samples=num_samples, max_length=max_length)\n",
    "    display_generated_verses(verses, prompt)\n",
    "    \n",
    "    # Add quality evaluation for the first verse\n",
    "    if verses:\n",
    "        quality = generator.evaluate_verse_quality(verses[0])\n",
    "        print(\"Quality metrics for first verse:\")\n",
    "        for metric, score in quality.items():\n",
    "            print(f\"- {metric}: {score:.2f}\")\n",
    "\n",
    "# Create interactive widgets\n",
    "interact(\n",
    "    generate_interactive,\n",
    "    prompt=widgets.Text(value=\"Trăm năm\", description=\"Starting phrase:\"),\n",
    "    num_samples=widgets.IntSlider(min=1, max=5, step=1, value=3, description=\"Number of verses:\"),\n",
    "    max_length=widgets.IntSlider(min=30, max=100, step=10, value=50, description=\"Max length:\")\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Themed Verse Generation\n",
    "\n",
    "Generate verses related to specific themes from Truyện Kiều."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_themed_verses(theme, num_samples=3):\n",
    "    # Get thematic elements from cultural context\n",
    "    elements = enhancer.get_thematic_elements(theme)\n",
    "    \n",
    "    # Print theme information\n",
    "    print(f\"Theme: {theme}\")\n",
    "    if 'theme_description' in elements:\n",
    "        print(f\"Description: {elements['theme_description']}\")\n",
    "    if elements['symbols']:\n",
    "        print(f\"Related symbols: {', '.join(elements['symbols'])}\")\n",
    "    if elements['characters']:\n",
    "        print(f\"Related characters: {', '.join(elements['characters'])}\")\n",
    "    print()\n",
    "    \n",
    "    # Generate verses with the theme\n",
    "    verses = generator.generate_with_theme(theme, num_samples=num_samples)\n",
    "    display_generated_verses(verses)\n",
    "\n",
    "# Create interactive theme selection\n",
    "interact(\n",
    "    generate_themed_verses,\n",
    "    theme=widgets.Dropdown(\n",
    "        options=['love', 'nature', 'fate', 'beauty', 'sadness', 'happiness'],\n",
    "        value='love',\n",
    "        description='Theme:'\n",
    "    ),\n",
    "    num_samples=widgets.IntSlider(min=1, max=5, step=1, value=3, description=\"Number of verses:\")\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Creating Verse Pairs\n",
    "\n",
    "Generate pairs of verses that work together poetically, as is common in Truyện Kiều."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_verse_pairs(starting_phrase=None, num_pairs=3):\n",
    "    html = \"<div style='background-color:#f9f9f9; padding:15px; border-radius:5px;'>\"\n",
    "    html += \"<h3>Generated verse pairs:</h3>\"\n",
    "    \n",
    "    for i in range(num_pairs):\n",
    "        first_verse, second_verse = generator.generate_verse_pair(starting_phrase)\n",
    "        \n",
    "        html += f\"<div style='margin-bottom:15px; padding:10px; border-left:3px solid #5a7d9a;'>\"\n",
    "        html += f\"<p><b>Pair {i+1}</b></p>\"\n",
    "        html += f\"<p>{first_verse}</p>\"\n",
    "        html += f\"<p>{second_verse}</p>\"\n",
    "        html += \"</div>\"\n",
    "    \n",
    "    html += \"</div>\"\n",
    "    display(HTML(html))\n",
    "\n",
    "# Create interactive verse pair generator\n",
    "interact(\n",
    "    generate_verse_pairs,\n",
    "    starting_phrase=widgets.Text(value=\"\", description=\"Starting phrase (optional):\"),\n",
    "    num_pairs=widgets.IntSlider(min=1, max=5, step=1, value=3, description=\"Number of pairs:\")\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Enhanced Verse Generation with Cultural Context\n",
    "\n",
    "Use the cultural context enhancer to create more culturally authentic verses."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_with_context(character_or_symbol, num_samples=3):\n",
    "    # Get contextual information\n",
    "    context_info = enhancer.get_contextual_info(character_or_symbol)\n",
    "    \n",
    "    if context_info:\n",
    "        # Print context information\n",
    "        print(f\"Found information about: {context_info['term']}\")\n",
    "        print(f\"Category: {context_info['category']}\")\n",
    "        print(f\"Description: {context_info['info']}\")\n",
    "        print()\n",
    "        \n",
    "        # Generate verse starting with the term\n",
    "        verses = generator.generate_verse(context_info['term'], num_samples=num_samples)\n",
    "        display_generated_verses(verses, context_info['term'])\n",
    "        \n",
    "        # Get pairing suggestions\n",
    "        if verses:\n",
    "            suggestions = enhancer.suggest_verse_pairings(verses[0])\n",
    "            if suggestions:\n",
    "                print(\"\\nCultural context suggestions:\")\n",
    "                for suggestion in suggestions:\n",
    "                    print(f\"- {suggestion}\")\n",
    "    else:\n",
    "        print(f\"No specific context found for '{character_or_symbol}'\")\n",
    "        # Generate verse anyway\n",
    "        verses = generator.generate_verse(character_or_symbol, num_samples=num_samples)\n",
    "        display_generated_verses(verses, character_or_symbol)\n",
    "\n",
    "# Create interactive cultural context generator\n",
    "interact(\n",
    "    generate_with_context,\n",
    "    character_or_symbol=widgets.Dropdown(\n",
    "        options=['thúy kiều', 'kim trọng', 'từ hải', 'trăng', 'hoa', 'đàn', 'liễu', 'mệnh'],\n",
    "        value='thúy kiều',\n",
    "        description='Cultural element:'\n",
    "    ),\n",
    "    num_samples=widgets.IntSlider(min=1, max=5, step=1, value=3, description=\"Number of verses:\")\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Conclusion\n",
    "\n",
    "In this notebook, we've demonstrated how to use our trained language models to generate verses in the style of Truyện Kiều. We've seen:\n",
    "\n",
    "1. Basic verse generation from prompts\n",
    "2. Interactive verse generation\n",
    "3. Theme-based verse generation\n",
    "4. Creation of verse pairs\n",
    "5. Culturally-enhanced verse generation\n",
    "\n",
    "The quality of the generated verses depends on the model used (n-gram vs. LSTM) and the amount of training data. The LSTM model generally produces more coherent verses when properly trained, especially on a GPU."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}